In [27]:
from typing import Tuple
from copy import deepcopy

A six-sided die, with numbers written on each of its faces, is placed on the 6-by-6 grid above, in the lower-left (yellow) corner. It then makes a sequence of “moves”. Each move consists of tipping the die into an orthogonally adjacent square within the grid.

The die starts with a “score” of 0. On the Nth move, its score increases by N times the value of the die facing up after the move. However, the die is only allowed to move into a square if its score after the move matches the value in the square. Also, the die cannot be translated or rotated in place in addition to these moves.

After some number of moves the die arrives in the upper-right (blue) corner.

The answer to this puzzle is the sum of values in the unvisited squares from the die’s journey.

In [46]:
grid = [[0, 77, 32, 403, 337, 452],
        [5, 23, -4, 592, 445, 620],
        [-7, 2, 357, 452, 317, 395],
        [186, 42, 195, 704, 452, 228], 
        [81, 123, 240, 443, 353, 508],
        [57, 33, 132, 268, 492, 732]]

initial_state = {'x': 0, 
                 'y': 0, 
                 'moves_made': 0, 
                 'orientation': (1,2,3), 
                 'revealed_sides': {}, 
                 'trajectory': [(0,0)]}

In [104]:
def roll(die_orientation: Tuple[int, int, int], direction: Tuple[int, int]) -> Tuple[int, int, int]:
    ''' 
    Given an initial die orientation + direction, return new die orientation. 
    Die orientation is defined as side on top, side on right, side on front.
    direction is one of {1, 2, 3, 4}
    '''
    top, right, front = die_orientation
    assert direction in [(0, 1), (0, -1), (1, 0), (-1, 0)]
    ## roll forward:
    if direction == (1, 0): 
        return 7 - front, right, top
    ## roll back:
    if direction == (-1, 0):
        return front, right, 7 - top
    ## roll right
    if direction == (0, 1):
        return 7 - right, top, front
    ## roll left
    if direction == (0, -1):
        return right, 7 - top, front

In [112]:
def is_final(state):
    return (state['x'], state['y']) == (5,5)

def possible_moves(state):
    moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    # make sure we stay in grid:
    if state['x'] == 0: moves = [m for m in moves if m[0] != -1]
    if state['x'] == 5: moves = [m for m in moves if m[0] != 1]
    if state['y'] == 0: moves = [m for m in moves if m[1] != -1]
    if state['y'] == 5: moves = [m for m in moves if m[1] != 1]
    return moves

def transition(state, direction):
    '''
    roll on direction. Returns bool "success" and new state
    ''' 
    top, right, front = roll(state['orientation'], direction)
    new_x = state['x'] + direction[0]
    new_y = state['y'] + direction[1]
    nth_move = state['moves_made'] + 1
    new_state = deepcopy(state)
    new_state['orientation'] = (top, right, front)
    new_state['x'] = new_x
    new_state['y'] = new_y
    new_state['moves_made'] = nth_move
    new_state['trajectory'] = [(new_x, new_y)] + new_state['trajectory']
    diff_score = grid[new_x][new_y] - grid[state['x']][state['y']]
    if top in new_state['revealed_sides']:
        if new_state['revealed_sides'][top] * nth_move == diff_score:
            success = True
        else:
            success = False
    else:
        if diff_score % nth_move == 0:
            assert float.is_integer(diff_score / nth_move), new_state
            new_state['revealed_sides'][top] = int(diff_score / nth_move)
            success = True
        else:
            success = False
    return new_state, success

In [119]:
reached_states = [initial_state]
reached_final_tile = False
while True:
    new_reached_states = []
    for state in reached_states:
#         print(f'We have the following possible moves: {possible_moves(state)}')
        for direction in possible_moves(state):
            new_state, success = transition(state, direction)
#             print(f'tried out new state, with {success} success:')
#             print(new_state)
            if success:
                if is_final(new_state):
                    final_state = state
                    reached_final_tile = True
                    print('TADAA')
                new_reached_states.append(new_state)
    reached_states = new_reached_states
    print(f'We have {len(reached_states)} trajectories so far')
    for state in reached_states:
        print(f'    {state}')
    if reached_final_tile or len(reached_states) == 0:
        break
print(final_state)

We have 2 trajectories so far
    {'x': 0, 'y': 1, 'moves_made': 1, 'orientation': (5, 1, 3), 'revealed_sides': {5: 77}, 'trajectory': [(0, 1), (0, 0)]}
    {'x': 1, 'y': 0, 'moves_made': 1, 'orientation': (4, 2, 1), 'revealed_sides': {4: 5}, 'trajectory': [(1, 0), (0, 0)]}
We have 3 trajectories so far
    {'x': 1, 'y': 1, 'moves_made': 2, 'orientation': (4, 1, 5), 'revealed_sides': {5: 77, 4: -27}, 'trajectory': [(1, 1), (0, 1), (0, 0)]}
    {'x': 1, 'y': 1, 'moves_made': 2, 'orientation': (5, 4, 1), 'revealed_sides': {4: 5, 5: 9}, 'trajectory': [(1, 1), (1, 0), (0, 0)]}
    {'x': 2, 'y': 0, 'moves_made': 2, 'orientation': (6, 2, 4), 'revealed_sides': {4: 5, 6: -6}, 'trajectory': [(2, 0), (1, 0), (0, 0)]}
We have 7 trajectories so far
    {'x': 1, 'y': 2, 'moves_made': 3, 'orientation': (6, 4, 5), 'revealed_sides': {5: 77, 4: -27, 6: -9}, 'trajectory': [(1, 2), (1, 1), (0, 1), (0, 0)]}
    {'x': 1, 'y': 0, 'moves_made': 3, 'orientation': (1, 3, 5), 'revealed_sides': {5: 77, 4: -27, 1

In [144]:
all_tiles = set([(a,b) for a in range(6) for b in range(6)])
trajectory_tiles = set(final_state['trajectory'])
unused_tiles = all_tiles - trajectory_tiles

In [141]:
all_tiles = set([(a,b) for a in range(5) for b in range(5)])

In [146]:
unused_tiles

{(1, 4), (2, 0), (3, 4), (3, 5), (5, 0), (5, 3), (5, 4), (5, 5)}

In [147]:
sum([grid[tile[0]][tile[1]] for tile in unused_tiles if tile != (5,5)])

1935

In [134]:
prev_value = 0
for step, tile in enumerate(final_state['trajectory'][::-1]):
    new_value = grid[tile[0]][tile[1]]
    print(f'for step {step}, we went to tile {tile} with value {new_value} and diff {new_value - prev_value}')
    prev_value = new_value

for step 0, we went to tile (0, 0) with value 0 and diff 0
for step 1, we went to tile (1, 0) with value 5 and diff 5
for step 2, we went to tile (1, 1) with value 23 and diff 18
for step 3, we went to tile (1, 2) with value -4 and diff -27
for step 4, we went to tile (0, 2) with value 32 and diff 36
for step 5, we went to tile (0, 1) with value 77 and diff 45
for step 6, we went to tile (1, 1) with value 23 and diff -54
for step 7, we went to tile (2, 1) with value 2 and diff -21
for step 8, we went to tile (3, 1) with value 42 and diff 40
for step 9, we went to tile (4, 1) with value 123 and diff 81
for step 10, we went to tile (5, 1) with value 33 and diff -90
for step 11, we went to tile (5, 2) with value 132 and diff 99
for step 12, we went to tile (4, 2) with value 240 and diff 108
for step 13, we went to tile (4, 1) with value 123 and diff -117
for step 14, we went to tile (4, 0) with value 81 and diff -42
for step 15, we went to tile (3, 0) with value 186 and diff 105
for step 